In [24]:
import pandas as pd

## DATAFRAME

In [25]:
df_BANK_marketing = pd.read_parquet(r"C:\Users\34622\Desktop\ProjecteData\Equip_C\Scripts\df_BANK_marketing.parquet")
df_BANK_marketing

,id,age,job,marital,education,is_in_default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,1,59.0,admin.,married,secondary,0,2343,1,0,unknown,5,may,1042,1,-1,0,unknown,1
1,2,59.0,admin.,married,secondary,0,2343,1,0,unknown,5,may,1042,1,-1,0,unknown,1
2,3,56.0,admin.,married,secondary,0,45,0,0,unknown,5,may,1467,1,-1,0,unknown,1
3,4,41.0,technician,married,secondary,0,1270,1,0,unknown,5,may,1389,1,-1,0,unknown,1
4,5,55.0,services,married,secondary,0,2476,1,0,unknown,5,may,579,1,-1,0,unknown,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11158,11159,33.0,blue-collar,single,primary,0,1,1,0,cellular,20,apr,257,1,-1,0,unknown,0
11159,11160,39.0,services,married,secondary,0,733,0,0,unknown,16,jun,83,4,-1,0,unknown,0
11160,11161,32.0,technician,single,secondary,0,29,0,0,cellular,19,aug,156,2,-1,0,unknown,0
11161,11162,43.0,technician,married,secondary,0,0,0,1,cellular,8,may,9,2,172,5,failure,0


## 1) KPI Conversion Deposito

In [26]:
deposito=df_BANK_marketing['deposit']
df_deposito=pd.DataFrame(deposito, columns=['deposit']).reset_index()
df_deposito
conversion_deposito = (df_deposito[df_deposito['deposit']==1].count()/df_deposito['deposit'].count())*100
conversion_deposito.round(2)

index      47.39
deposit    47.39
dtype: float64

## 2) Promedio de la duración de las llamadas de los suscriptores

In [27]:
## número de suscriptores
suscriptores = df_BANK_marketing[df_BANK_marketing['deposit']==1].count()
num_suscriptores = suscriptores[['duration']]
num_suscriptores

duration    5290
dtype: int64

In [28]:
## sumatorio de la duracion de las  llamadas hechas a suscriptores
sumatorio_duracion_llamadas = suscriptores['duration'].sum().round(2)
sumatorio_duracion_llamadas

5290

In [29]:
## promedio de las llamadas a suscriptores
promedio_llamadas_suscriptores = (sumatorio_duracion_llamadas/num_suscriptores).round(2)
promedio_llamadas_suscriptores

duration    1.0
dtype: float64

## 3) Porcentage de llamadas a teléfono fijo o a móvil

In [30]:
## número de llamadas a móvil y teléfono
contacto = df_BANK_marketing['contact']
contacto.count()
df_contacto=pd.DataFrame(contacto, columns=['contact']).reset_index()
df_contacto
llamadas = df_contacto[(df_contacto['contact'] == 'cellular')|(df_contacto['contact'] == 'telephone')].count()
llamadas

index      8816
contact    8816
dtype: int64

# ?? NUMERO DE LLAMADAS TOTALES = SON CAMPAIGN???

In [31]:
porcentage = ((llamadas/(contacto.count()))*100).round(2)
porcentage

index      78.98
contact    78.98
dtype: float64

## 4) Promedio de contactos previos de suscriptores

In [32]:
## sumatorio del número de contactos previos de suscriptores
previous = df_BANK_marketing[['previous']].sum()
previous

previous    9293
dtype: int64

In [33]:
suscriptores[['previous']]

previous    5290
dtype: int64

In [34]:
promedio_contactos_previos = (df_BANK_marketing[['previous']].sum())/suscriptores[['previous']]
promedio_contactos_previos.round(2)

previous    1.76
dtype: float64

## 5) Mes con mayor tasa de conversión

In [47]:
df_BANK_marketing[['month','campaign', 'deposit']]

,month,campaign,deposit
0,may,1,1
1,may,1,1
2,may,1,1
3,may,1,1
4,may,1,1
...,...,...,...
11158,apr,1,0
11159,jun,4,0
11160,aug,2,0
11161,may,2,0


In [61]:
## Número de suscriptores en el mes
num_susc = df_BANK_marketing.groupby(['month']).deposit.sum().sort_values(ascending=False)
df_num_susc = pd.DataFrame(num_susc, columns =['deposit']).reset_index()
df_num_susc = df_num_susc.rename(columns = {'deposit':'num_suscriptores'})
df_num_susc

,month,num_suscriptores
0,may,926
1,aug,688
2,jul,627
3,apr,577
4,jun,546
5,feb,441
6,nov,403
7,oct,323
8,sep,269
9,mar,248


In [60]:
# Número de clientes contactados en el mes
num_cliente_cont = df_BANK_marketing.groupby(['month']).deposit.count().sort_values(ascending=False)
df_num_cliente_cont = pd.DataFrame(num_cliente_cont, columns =['deposit']).reset_index()
df_num_cliente_cont = df_num_cliente_cont.rename(columns = {'deposit':'num_clientes_cont'})
df_num_cliente_cont

,month,num_clientes_cont
0,may,2825
1,aug,1519
2,jul,1514
3,jun,1222
4,nov,943
5,apr,923
6,feb,776
7,oct,392
8,jan,344
9,sep,319


In [81]:
## concatenar las dos tablas
concatenar = pd.merge(df_num_susc,df_num_cliente_cont, on='month')
concatenar['dividir'] = ((concatenar['num_suscriptores']/concatenar['num_clientes_cont'])*100).round(2)
df_ordenar = concatenar.sort_values(by='dividir', ascending=False)
df_ordenar.head(1)

,month,num_suscriptores,num_clientes_cont,dividir
11,dec,100,110,90.91
